<table align="left">
  <td>
    <a href="https://colab.research.google.com/github/marco-canas/intro-Machine-Learning/blob/main/classes/class_18/class_18_pipeline_linear_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
  </td>
</table> 

# Clase 18 

In [1]:
import pandas as pd 
v = pd.read_csv('vivienda.csv')

In [2]:
v.head()

,longitud,latitud,antiguedad,habitaciones,dormitorios,población,hogares,ingresos,proximidad,valor
0,-122.23,37.88,41.0,880.0,129.0,322.0,126.0,8.3252,NEAR BAY,452600.0
1,-122.22,37.86,21.0,7099.0,1106.0,2401.0,1138.0,8.3014,NEAR BAY,358500.0
2,-122.24,37.85,52.0,1467.0,190.0,496.0,177.0,7.2574,NEAR BAY,352100.0
3,-122.25,37.85,52.0,1274.0,235.0,558.0,219.0,5.6431,NEAR BAY,341300.0
4,-122.25,37.85,52.0,1627.0,280.0,565.0,259.0,3.8462,NEAR BAY,342200.0


# Sintesis de la información

In [3]:
v.info() 


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20640 entries, 0 to 20639
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   longitud      20640 non-null  float64
 1   latitud       20640 non-null  float64
 2   antiguedad    20640 non-null  float64
 3   habitaciones  20640 non-null  float64
 4   dormitorios   20433 non-null  float64
 5   población     20640 non-null  float64
 6   hogares       20640 non-null  float64
 7   ingresos      20640 non-null  float64
 8   proximidad    20640 non-null  object 
 9   valor         20640 non-null  float64
dtypes: float64(9), object(1)
memory usage: 1.6+ MB


In [ ]:
v.proximidad.value_counts() 

In [ ]:
v.describe() 

# Visualizaciones 

In [ ]:
v.hist(figsize = (15,10)) 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 1, figsize = (10,6)) 

In [ ]:
v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1, figsize = (10,6)) 

In [ ]:
import matplotlib.pyplot as plt 

v.plot(kind = 'scatter', x = 'longitud', y = 'latitud', alpha = 0.1, figsize = (10,6),\
      s = v.población/100, label = 'Población', \
      c = 'valor', cmap = plt.get_cmap('jet'), colorbar = True)  

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
v_train, v_test = train_test_split(v, test_size = 0.2, random_state = 42) 

In [6]:
v = v_train 

# Buscando correlaciones

Dado que el conjunto de datos no es demasiado grande, puede calcular fácilmente el coeficiente de correlación estándar (también llamado r de Pearson) entre cada par de atributos usando el método `corr()`:

In [7]:
v.corr() 

,longitud,latitud,antiguedad,habitaciones,dormitorios,población,hogares,ingresos,valor
longitud,1.000000,-0.924485,-0.101818,0.038676,0.063064,0.094276,0.049306,-0.017040,-0.046349
latitud,-0.924485,1.000000,0.005296,-0.029224,-0.059998,-0.102499,-0.064061,-0.076571,-0.142983
antiguedad,-0.101818,0.005296,1.000000,-0.360922,-0.320624,-0.292283,-0.302796,-0.121711,0.103706
habitaciones,0.038676,-0.029224,-0.360922,1.000000,0.930489,0.857936,0.920482,0.198268,0.133989
dormitorios,0.063064,-0.059998,-0.320624,0.930489,1.000000,0.878932,0.980255,-0.009141,0.047980
población,0.094276,-0.102499,-0.292283,0.857936,0.878932,1.000000,0.907452,0.004122,-0.026032
hogares,0.049306,-0.064061,-0.302796,0.920482,0.980255,0.907452,1.000000,0.012776,0.063714
ingresos,-0.017040,-0.076571,-0.121711,0.198268,-0.009141,0.004122,0.012776,1.000000,0.690647
valor,-0.046349,-0.142983,0.103706,0.133989,0.047980,-0.026032,0.063714,0.690647,1.000000


Ahora veamos cómo cada atributo está correlacionado con la variable objetivo de valor promedio de vivienda por distrito: 

In [8]:
v.corr().valor.sort_values(ascending = False) 

valor           1.000000
ingresos        0.690647
habitaciones    0.133989
antiguedad      0.103706
hogares         0.063714
dormitorios     0.047980
población      -0.026032
longitud       -0.046349
latitud        -0.142983
Name: valor, dtype: float64

# Crear el DataFrame numérico y el DataFrame categórico

In [ ]:
v_num = v.drop('proximidad', axis = 1) 

In [ ]:
v_cat = v.proximidad 

1. Imputar los datos faltantes

In [ ]:
from sklearn.impute import SimpleImputer

In [ ]:
imputar = SimpleImputer(strategy = 'median') 

In [ ]:
imputar.fit(v_num)

In [ ]:
X = imputar.transform(v_num) 

In [ ]:
v_num_imputado = pd.DataFrame(X, columns = v_num.columns, index = v_num.index) 

In [ ]:
v_num_imputado.info() 

In [ ]:
len(v_num_imputado) 

In [ ]:
v = v_num_imputado 

## 2. establecimiento de correlaciones. 

<img src = 'https://github.com/marco-canas/taca/blob/main/ref/geron/part_1/chap_2/4_visualize/figura_2_14_Standard_correlation_coefficient_of_various_datasets.PNG?raw=true'>

In [ ]:
v.corr() 

In [ ]:
v.corr().valor.sort_values(ascending =False)  

In [ ]:
v['dormitorios_por_habitaciones'] = v.dormitorios/v.habitaciones 

In [ ]:
v['población_por_hogares'] = v.población/v.hogares 

In [ ]:
v['habitaciones_por_hogar'] = v.habitaciones/v.hogares 

In [ ]:
v.corr().valor.sort_values(ascending = False) 

## 2. codificación de datos categóricos

In [ ]:
v_cat 

3. standarizar los datos